## 1. Module import 및 환경 변수 설정

In [ ]:
## Import modules
import pandas as pd
import numpy as np

import os
#from datetime import datetime, timedelta

In [ ]:
## Input data 경로설정
curr_dir = os.getcwd()
print(curr_dir)

os.chdir("/home/sookyoung.kim/ssd/projects/GraphDB/customer_index/complaint_index/w12_1-12_1_at_once/test_vectors_gt_cx2_tra")
#os.system('ls')
train_data_path = os.getcwd()
print(train_data_path)
os.chdir(curr_dir)

os.chdir("/home/sookyoung.kim/ssd/projects/GraphDB/customer_index/complaint_index/w12_1-12_1_at_once/test_vectors_gt_cx2_inf")
#os.system('ls')
infer_data_path = os.getcwd()
print(infer_data_path)
os.chdir(curr_dir)

## Input file 명 설정
train_data = os.path.join(train_data_path, "vectors_CUST_NO.csv")
print(train_data)

## Input file 명 설정
infer_data = os.path.join(infer_data_path, "vectors_CUST_NO.csv")
print(infer_data)

## 2. Regression을 위한 입력 data 생성

In [ ]:
import ast
#from ast import literal_eval

vdim = 256
vcol_name = 'embedding_'+str(vdim)
print(vcol_name)

df_train = pd.read_csv(train_data)
list_index = df_train.index

list_dimensions = []
for i in range(0, vdim):
    el = 'D'+str(i).zfill(3)
    list_dimensions.append(el)

# Each vector is stored in string. Convert each vector to list first.
series_vectors = df_train[vcol_name].apply(lambda x: ast.literal_eval(x))
# Then, make the vectors 2D list
list_vectors = series_vectors.to_list()

df_vectors = pd.DataFrame(data=list_vectors, index=list_index, columns=list_dimensions)

df_train.drop([vcol_name], axis=1, inplace=True)

df_vectors['CUST_NO'] = df_train['CUST_NO']

df_train = pd.merge(df_train, df_vectors, how='left', on=['CUST_NO'])

df_train.to_csv('df_train.csv', index=False)

In [ ]:
import ast
#from ast import literal_eval

vdim = 256
vcol_name = 'embedding_'+str(vdim)
print(vcol_name)

df_infer = pd.read_csv(infer_data)
list_index = df_infer.index

list_dimensions = []
for i in range(0, vdim):
    el = 'D'+str(i).zfill(3)
    list_dimensions.append(el)

# Each vector is stored in string. Convert each vector to list first.
series_vectors = df_infer[vcol_name].apply(lambda x: ast.literal_eval(x))
# Then, make the vectors 2D list
list_vectors = series_vectors.to_list()

df_vectors = pd.DataFrame(data=list_vectors, index=list_index, columns=list_dimensions)

df_infer.drop([vcol_name], axis=1, inplace=True)

df_vectors['CUST_NO'] = df_infer['CUST_NO']

df_infer = pd.merge(df_infer, df_vectors, how='left', on=['CUST_NO'])

df_infer.to_csv('df_infer.csv', index=False)

In [ ]:
df_train_infer = pd.concat([df_train, df_infer], ignore_index=True)

In [ ]:
# df_train_infer 에서  
# df_train_noclaim
# df_train_claim
# df_infer_noclaim
# df_infer_claim
# df_train_infer_noclaim
# df_train_infer_claim
# 으로 나눔

cond_tmp = ((df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 0) | (df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)) & \
            (df_train_infer.loc[:, 'LABEL_CLAIM'] == 0)
df_train_noclaim = df_train_infer.loc[cond_tmp, :]
cond_tmp = ((df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 0) | (df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)) & \
            (df_train_infer.loc[:, 'LABEL_CLAIM'] == 1)
df_train_claim = df_train_infer.loc[cond_tmp, :]

cond_tmp = ((df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 1) | (df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)) & \
            (df_train_infer.loc[:, 'LABEL_CLAIM'] == 0)
df_infer_noclaim = df_train_infer.loc[cond_tmp, :]
cond_tmp = ((df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 1) | (df_train_infer.loc[:, 'FLAG_TRAIN_INFER'] == 2)) & \
            (df_train_infer.loc[:, 'LABEL_CLAIM'] == 1)
df_infer_claim = df_train_infer.loc[cond_tmp, :]

cond_tmp = (df_train_infer.loc[:, 'LABEL_CLAIM'] == 0)
df_train_infer_noclaim = df_train_infer.loc[cond_tmp, :]

cond_tmp = (df_train_infer.loc[:, 'LABEL_CLAIM'] == 1)
df_train_infer_claim = df_train_infer.loc[cond_tmp, :]

In [ ]:
df_train_infer_claim.head()

## 3. Train set과 inference set 간 claim/noclaim 샘플들 간 유사도 확인

1시간 이상 걸립니다. Plotting해 상태를 보기 위함이니 생략해도 됩니다.

In [ ]:
import umap.umap_ as umap

# UMAP dimension reduction from 256 to 2
def umap_dimension_reduction_d256_to_d2(df_h, n_neighbors, min_dist, metric, output_df_col_list):
        
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2, metric=metric, verbose=True)
       
    list_header = []
    for i in range(0, 256):
        el = 'D'+str(i).zfill(3)
        list_header.append(el)
    
    hd_vector_data = df_h[list_header].values
    ld_vector_data = reducer.fit_transform(hd_vector_data)
    
    df_l = pd.DataFrame(data = ld_vector_data.tolist(), columns = output_df_col_list, index=df_h.index)
    
    return df_l

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(data=df_train_infer_claim_vector_ld, 
                x='D1', y='D2', 
                hue='FLAG_TRAIN_INFER', palette='muted').set_title('Train Vs. Inference clusters');
plt.ylim(-10, 20)
plt.xlim(-4, 14)

In [ ]:
# UMAP을 이용한 graph embedded vectors (256차원)의 차원 축소 (2차원)
df_train_infer_noclaim_vector = df_train_infer_noclaim.copy()
df_train_infer_noclaim_vector.drop(['CUST_NO'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['CX_TYPE'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['FLAG_TRAIN_INFER'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['LABEL_CLAIM'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['LABEL_CLAIM_INF'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['LABEL_CLAIM_FUTURE'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['LABEL_CLAIM_NEXT'], axis=1, inplace=True)
df_train_infer_noclaim_vector.drop(['RCPT_DTHM'], axis=1, inplace=True)

df_train_infer_noclaim_vector_ld = umap_dimension_reduction_d256_to_d2(df_train_infer_noclaim_vector, 50, 0.1, 'euclidean', ['D1','D2'])

df_train_infer_noclaim_vector_ld['FLAG_TRAIN_INFER'] = df_train_infer_noclaim['FLAG_TRAIN_INFER']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.scatterplot(data=df_train_infer_noclaim_vector_ld, 
                x='D1', y='D2', 
                hue='FLAG_TRAIN_INFER', palette='muted').set_title('Train Vs. Inference clusters');
plt.ylim(-10, 20)
plt.xlim(-4, 14)

## 3. Distance-based down sampling

위에서 vector들의 분포 양상을 확인한 결과, label 1과 0인 sample들이 겹쳤기에, train set으로 학습해도 inference set을 제대로 예측하지 못할 가능성이 높습니다.

따라서, train set의 label 1인 sample들과 의도적으로 거리가 먼 label 0 sample들만 추출해 학습시키는 negative sampling을 하는 과정입니다.

#### Train set 내의 claim samples의 자신들의 centroid로부터의 거리

In [ ]:
#from sklearn.neighbors.nearest_centroid import NearestCentroid
import math

# Claim samples의 centroid 찾기
list_train_claim_centroid = df_train_claim[list_dimensions].mean().to_list()
train_claim_centroid = np.array(list_train_claim_centroid)
#print(list_train_claim_centroid)
#print(sum(list_train_claim_centroid) / len(list_train_claim_centroid))

# Noclaim samples와의 거리 계산
df_train_claim_dist = df_train_claim[list_dimensions]
df_train_claim_dist['DIST'] = df_train_claim_dist.sub(train_claim_centroid, axis=1).pow(2).sum(axis=1).pow(.5)

train_claim_dist_max = df_train_claim_dist['DIST'].max()
train_claim_dist_min = df_train_claim_dist['DIST'].min()
print(train_claim_dist_max, train_claim_dist_min)

df_train_claim_dist['CUST_NO'] = df_train_claim['CUST_NO']
df_train_claim_dist['CX_TYPE'] = df_train_claim['CX_TYPE']
df_train_claim_dist['FLAG_TRAIN_INFER'] = df_train_claim['FLAG_TRAIN_INFER']
df_train_claim_dist['LABEL_CLAIM'] = df_train_claim['LABEL_CLAIM']
df_train_claim_dist['LABEL_CLAIM_INF'] = df_train_claim['LABEL_CLAIM_INF']
df_train_claim_dist['LABEL_CLAIM_FUTURE'] = df_train_claim['LABEL_CLAIM_FUTURE']
df_train_claim_dist['LABEL_CLAIM_NEXT'] = df_train_claim['LABEL_CLAIM_NEXT']
df_train_claim_dist['RCPT_DTHM'] = df_train_claim['RCPT_DTHM']

#df_train_claim_dist = df_train_claim_dist.sort_values(by=['DIST'], ascending=False)
df_train_claim_dist = df_train_claim_dist.sort_values(by=['DIST'], ascending=True)
print(df_train_claim_dist['DIST'])
df_train_claim_dist.reset_index(drop=True, inplace=True)
#df_train_claim_dist.drop(['DIST'], axis=1, inplace=True)

# Down sampling noclaim
len_train_claim = len(df_train_claim)
print(len_train_claim)

'''
df_train_claim_undersamples = df_train_claim_dist.iloc[0:len_train_claim*3,:]
#print(df_train_noclaim_undersamples)

df_train_claim_undersamples.drop(['DIST'], axis=1, inplace=True)

# Make up the train set
df_train_undersamples = pd.concat([df_train_claim, df_train_claim_undersamples], ignore_index=True)
#print(df_train_claim_undersamples)
#print(df_train_undersamples)
#df_train_undersamples.to_csv('vectors_CUST_NO_under.csv')
'''

In [ ]:
list_train_noclaim_centroid = df_train_noclaim[list_dimensions].mean().to_list()
train_noclaim_centroid = np.array(list_train_noclaim_centroid)
print(list_train_noclaim_centroid)
print(sum(list_train_noclaim_centroid) / len(list_train_noclaim_centroid))

#### Train set 내의 noclaim samples의 claim samples centroid로부터의 거리

In [ ]:
import math

### Euclidean distance

# Claim samples의 centroid 찾기
list_train_claim_centroid = df_train_claim[list_dimensions].mean().to_list()
train_claim_centroid = np.array(list_train_claim_centroid)

# Noclaim samples와의 거리 계산
df_train_noclaim_dist = df_train_noclaim[list_dimensions]
df_train_noclaim_dist['DIST'] = df_train_noclaim_dist.sub(train_claim_centroid, axis=1).pow(2).sum(axis=1).pow(.5)

df_train_noclaim_dist['CUST_NO'] = df_train_noclaim['CUST_NO']
df_train_noclaim_dist['CX_TYPE'] = df_train_noclaim['CX_TYPE']
df_train_noclaim_dist['FLAG_TRAIN_INFER'] = df_train_noclaim['FLAG_TRAIN_INFER']
df_train_noclaim_dist['LABEL_CLAIM'] = df_train_noclaim['LABEL_CLAIM']
df_train_noclaim_dist['LABEL_CLAIM_INF'] = df_train_noclaim['LABEL_CLAIM_INF']
df_train_noclaim_dist['LABEL_CLAIM_FUTURE'] = df_train_noclaim['LABEL_CLAIM_FUTURE']
df_train_noclaim_dist['LABEL_CLAIM_NEXT'] = df_train_noclaim['LABEL_CLAIM_NEXT']
df_train_noclaim_dist['RCPT_DTHM'] = df_train_noclaim['RCPT_DTHM']

df_train_noclaim_dist_gt = df_train_noclaim_dist.sort_values(by=['DIST'], ascending=False)
df_train_noclaim_dist_lt = df_train_noclaim_dist.sort_values(by=['DIST'], ascending=True)
print(df_train_noclaim_dist_gt['DIST'])
df_train_noclaim_dist_gt.reset_index(drop=True, inplace=True)
print(df_train_noclaim_dist_lt['DIST'])
df_train_noclaim_dist_lt.reset_index(drop=True, inplace=True)

# Down sampling noclaim
len_train_claim = len(df_train_claim)
print(len_train_claim)

if 1:
    df_train_noclaim_undersamples_lt = df_train_noclaim_dist_lt.iloc[0:round(len_train_claim*2.5),:]
    df_train_noclaim_undersamples_gt = df_train_noclaim_dist_gt.iloc[0:round(len_train_claim*2.5),:]
else:
    cond_tmp = df_train_noclaim_dist.loc[:, 'DIST'] < train_claim_dist_min
    df_train_noclaim_undersamples_lt = \
        df_train_noclaim_dist.loc[cond_tmp, :].sort_values(by=['DIST'], ascending=False).iloc[0:round(len_train_claim*1.5),:]
    cond_tmp = df_train_noclaim_dist.loc[:, 'DIST'] > train_claim_dist_max
    df_train_noclaim_undersamples_gt = \
        df_train_noclaim_dist.loc[cond_tmp, :].sort_values(by=['DIST'], ascending=True).iloc[0:round(len_train_claim*1.5),:]

df_train_noclaim_undersamples = pd.concat([df_train_noclaim_undersamples_lt, df_train_noclaim_undersamples_gt], ignore_index=True)

df_train_noclaim_undersamples.drop(['DIST'], axis=1, inplace=True)

# Make up the train set
df_train_undersamples = pd.concat([df_train_claim, df_train_noclaim_undersamples], ignore_index=True)
#print(df_train_noclaim_undersamples)
#print(df_train_undersamples)
df_train_undersamples.to_csv('vectors_CUST_NO_under.csv')

## 4. LGBM regression

In [ ]:
df_train_undersamples.head()

In [ ]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn import metrics

#x = df_train[list_dimensions]
#y = df_train[['LABEL_CLAIM']]
x = df_train_undersamples[list_dimensions]
y = df_train_undersamples[['LABEL_CLAIM']]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.99, test_size=0.01)

# Ensemble - Boosting - Random forest classification
lgbm_train_ds = lgbm.Dataset(x_train, label=y_train)
lgbm_test_ds = lgbm.Dataset(x_test, label=y_test)
#'''
lgbm_params = {
    'verbose': -1,
    'learning_rate': 0.01,
    'max_depth': 12,
    'num_leaves': 12,
    'objective': 'binary',
    'metric': 'mae',
    'boosting': 'gbdt',
}
#'''
'''
lgbm_params = {
    'verbose': -1,
    'learning_rate': 0.01,
    'max_depth': 31,
    'num_leaves': 31,
    'objective': 'regression',
    'metric': 'mae',
    'boosting': 'gbdt',
}
'''

lgbm_model = lgbm.train(params=lgbm_params,
                        train_set=lgbm_train_ds,
                        #num_boost_round=10000,
                        num_boost_round=5000,
                        valid_sets=lgbm_test_ds,
                        #verbose_eval=False,  # True, False, or a number
                        #early_stopping_rounds=1000,
                        )

df_fit = df_infer.loc[:, list_dimensions]

df_fit.loc[:, 'LABEL_CLAIM_PRED'] = lgbm_model.predict(df_fit)

In [ ]:
from sklearn.metrics import classification_report

df_fit['LABEL_CLAIM'] = df_infer.loc[:, 'LABEL_CLAIM']

y_true = df_fit['LABEL_CLAIM'].to_list()
#print(y_true)
y_pred = df_fit['LABEL_CLAIM_PRED'].to_list()
print(y_pred)

df_fit.loc[:, 'LABEL_CLAIM_PRED_ROUND'] = 0
cond_tmp = df_fit.loc[:, 'LABEL_CLAIM_PRED'] >= 0.5
df_fit.loc[cond_tmp, 'LABEL_CLAIM_PRED_ROUND'] = 1
y_pred = df_fit['LABEL_CLAIM_PRED_ROUND'].to_list()

#print(classification_report(y_true, y_pred, target_names=[0, 1]))
print(classification_report(y_true, y_pred))

confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
print(confusion_matrix)

#print(metrics.f1_score(y_true, y_pred, average='macro'))
#print(metrics.f1_score(y_true, y_pred, average='micro'))
#print(metrics.f1_score(y_true, y_pred, average='weighted'))

In [ ]:
from sklearn.metrics import classification_report

df_fit['LABEL_CLAIM'] = df_infer.loc[:, 'LABEL_CLAIM']

y_true = df_fit['LABEL_CLAIM'].to_list()
#print(y_true)
y_pred = df_fit['LABEL_CLAIM_PRED'].to_list()
print(y_pred)

df_fit.loc[:, 'LABEL_CLAIM_PRED_ROUND'] = 0
cond_tmp = df_fit.loc[:, 'LABEL_CLAIM_PRED'] >= 0.99
df_fit.loc[cond_tmp, 'LABEL_CLAIM_PRED_ROUND'] = 1
y_pred = df_fit['LABEL_CLAIM_PRED_ROUND'].to_list()

#print(classification_report(y_true, y_pred, target_names=[0, 1]))
print(classification_report(y_true, y_pred))

confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
print(confusion_matrix)

#print(metrics.f1_score(y_true, y_pred, average='macro'))
#print(metrics.f1_score(y_true, y_pred, average='micro'))
#print(metrics.f1_score(y_true, y_pred, average='weighted'))

In [ ]:
df_fit.head()